In [1]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    Settings,
)
from llama_parse import LlamaParse

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from dotenv import load_dotenv

# NESTED ASYNCIO LOOP NEEDED TO RUN ASYNC IN A NOTEBOOK
import nest_asyncio

nest_asyncio.apply()

# load the environment variables
load_dotenv()

# check if storage already exists
PERSIST_DIR = "./storage"

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")


# create the parser
parser = LlamaParse(result_type="markdown")

file_extractor = {".pdf": parser}

# load the documents
documents = SimpleDirectoryReader("data",file_extractor=file_extractor).load_data()

# chunk
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

if not os.path.exists(PERSIST_DIR):
    # create the index
    index = VectorStoreIndex.from_documents(documents, transformations=[text_splitter])
    
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

Started parsing the file under job_id 2904c442-3178-4c18-8b20-434bf38d55cd


In [2]:
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="llama3.1:latest", request_timeout=60.0,temperature=0.3)

In [3]:
query = "校園行事曆有甚麼活動？請詳細列出"

In [4]:
# Each will retrieve the top-2 most similar nodes
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever

# configure retriever
vector_retriever = index.as_retriever(similarity_top_k=2, verbose=True)
# vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3, verbose=True)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

resource module not available on Windows


In [5]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=3,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [6]:
from typing import List
from pydantic import BaseModel, Field


class AnswerOutput(BaseModel):
    """Output schema for the answer."""

    answer: str = Field(..., description="answer of the query")
    references: List[str] = Field(..., description="list of reference file path")

In [7]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import get_response_synthesizer

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
    
)
response = query_engine.query("一月有甚麼活動？")
print(response)

# file_names = [os.path.basename(path) for path in response.references]
# print(
#     f"response: \n answer_content: {response.answer}\n Reference_file_path: {file_names}"
# )
print("-----------------")
for i, node in enumerate(response.source_nodes):
    print(f"Node {i + 1}:")
    for key, value in node.metadata.items():
        print(f"{key}: {value}")
    print(f"content: {node.get_content()}")
    print("-----------------")

Generated queries:
Here are three search queries related to the input:
1. "一月的節日活動"
2. "台北一月的行事曆"
3. "全球一月的文化慶典"
清新宿舍健康日。
-----------------
Node 1:
file_path: c:\Users\User\Desktop\LamaIndex_RAG\data\165-未來大學-校園行事曆.pdf
file_name: 165-未來大學-校園行事曆.pdf
file_type: application/pdf
file_size: 73012
creation_date: 2024-11-19
last_modified_date: 2024-09-11
content: # 校園行事曆

# 一月

1月15日：清新宿舍健康日

地點：宿舍區健身房

內容：免費健康檢查、瑜伽課程、營養講座

# 二月

2月5日：國際文化節

地點：校園廣場

內容：各國文化展覽、傳統美食、文化表演

# 三月

3月8日：環保義工日

地點：校園及周邊社區

內容：垃圾清理、植樹活動、環保教育

3月15日：創業中心開放日

地點：創業中心

內容：創業指導、企業家講座、創業展示

# 四月

4月1日：清潔飲水與衛生設施宣導日

地點：清潔飲水設施區

內容：飲水衛生教育、節水設備展示

4月20日：多元課程大樓開放日

內容：課程介紹、學術講座、學生作品展覽

# 五月

5月5日：氣候變遷研討會

地點：氣候變遷研究中心

內容：氣候科學講座、政策討論、環保活動

# 六月

6月10日：全球發展合作論壇

地點：國際合作中心

內容：全球合作項目介紹、國際發展研討會

6月25日：運動會

地點：運動場

內容：田徑比賽、團隊運動、健身挑戰

# 七月

7月20日：社區服務日

地點：社區服務區

內容：志工服務、社區活動、文化交流

# 八月

8月25日：二手市集

地點：校園廣場

內容：二手物品交易、回收教育
-----------------
Node 2:
file_path: c:\Users\User\Desktop\LamaIndex_RAG\data\165-未來大學-校園行事曆.pdf
file_name: 1

In [8]:
prompts_dict = query_engine.get_prompts()
for k, p in prompts_dict.items():
    print(f"Prompt Key: {k}\nText:\n{p.get_template()}\n")


Prompt Key: response_synthesizer:text_qa_template
Text:
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 

Prompt Key: response_synthesizer:refine_template
Text:
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 



In [9]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

nodes_with_scores = retriever.retrieve(query)

for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

Generated queries:
Here are three search queries related to the input:
1. "校園行事曆活動列表"
2. "學校活動安排表格"
3. "大學校園活動一覽表"
Score: 0.13 - # 九月

# 9月5日：新學年開學典禮

地點：大禮堂

內容：校長致辭、新生介紹、校園生活指導

# 9月20日：學生權益講座

地點：學生權益保護機構

內容：學生權益保護講座、法律援助介紹

# 十月

# 10月10日：校園節能日

地點：校園各處

內容：節能設備展示、節能比賽

# 10月24日：國際交流活動

地點：國際合作中心

內容：留學生交流、國際文化展示

# 十一月

# 11月8日：科技創新大會

地點：科研與創新區

內容：科技創新展示、專題研討會

# 十二月

# 12月21日：聖誕慶祝活動

地點：校園廣場

內容：聖誕樹點燈、音樂表演、慈善義賣...
-----

Score: 0.13 - # 校園行事曆

# 一月

1月15日：清新宿舍健康日

地點：宿舍區健身房

內容：免費健康檢查、瑜伽課程、營養講座

# 二月

2月5日：國際文化節

地點：校園廣場

內容：各國文化展覽、傳統美食、文化表演

# 三月

3月8日：環保義工日

地點：校園及周邊社區

內容：垃圾清理、植樹活動、環保教育

3月15日：創業中心開放日

地點：創業中心

內容：創業指導、企業家講座、創業展示

# 四月

4月1日：清潔飲水與衛生設施宣導日

地點：清潔飲水設施區

內容：飲水衛生教育、節水設備展示

4月20日：多元課程大樓開放日

內容：課程介紹、學術講座、學生作品展覽

# 五月

5月5日：氣候變遷研討會

地點：氣候變遷研究中心

內容：氣候科學講座、政策討論、環保活動

# 六月

6月10日：全球發展合作論壇

地點：國際合作中心

內容：全球合作項目介紹、國際發展研討會

6月25日：運動會

地點：運動場

內容：田徑比賽、團隊運動、健身挑戰

# 七月

7月20日：社區服務日

地點：社區服務區

內容：志工服務、社區活動、文化交流

# 八月

8月25日：二手市集

地點：校園廣場

內容：二手物品交易、回收教育...
-----



In [10]:
# prompt
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in the style of a Shakespeare play.\n"
    "Query: {query_str}\n"
    "Answer: "
)

# prompt_tmpl = PromptTemplate(
#     qa_prompt_tmpl_str,
#     context_str="\n\n".join([n.get_content() for n in retrieved_nodes]),
#     query_str=query,
# )
# # configure response synthesizer
# response_synthesizer = get_response_synthesizer(text_qa_template=qa_prompt_tmpl_str)


query = "What is the capital of France?"

In [11]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondenseQuestionChatEngine


Condense_prompt = PromptTemplate(
"""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.
用中文回答

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
"""
)

# list of `ChatMessage` objects
custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content="一月有什麼活動？",
    ),
    ChatMessage(role=MessageRole.ASSISTANT, content="清新宿舍健康日."),
]

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=Condense_prompt,
    chat_history=custom_chat_history,
    verbose=True,
)

In [12]:
response = chat_engine.chat("我剛剛問了什麼?")
print(response)

Querying with: 一月份清新宿舍會舉辦什麼活動？
Generated queries:
Here are three search queries related to the input:
1. "一月份清新宿舍會活動安排"
2. "清新宿舍會一月份的主題活動"
3. "一月份清新宿舍會有哪些趣味活動"
健康日。


In [13]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondensePlusContextChatEngine

Condense_prompt = PromptTemplate(
    """\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.


<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
用中文產生
"""
)

# list of `ChatMessage` objects
custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content="一月有什麼活動？",
    ),
    ChatMessage(role=MessageRole.ASSISTANT, content="清新宿舍健康日."),
]

condense_plus_chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    chat_history=custom_chat_history,
    condense_question_prompt=Condense_prompt,
    verbose=True,
)

In [14]:
response = condense_plus_chat_engine.chat("我剛剛問什麼?")
print(response)

Condensed question: What activities are there in January?
Generated queries:
Here are three search queries related to the input:
1. "Things to do in January by country"
2. "January events and festivals worldwide"
3. "Popular outdoor activities in January"
你剛剛問了一月有什麼活動呢！根據校園行事曆，我們知道有一月15日的活動是清新宿舍健康日。除了這個活動之外，還沒有其他一月份的活動被列出來。


In [15]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from types import SimpleNamespace
# check if dataset already exists
PERSIST_DATASET_FILE = "./rag_dataset.json"

if not os.path.exists(PERSIST_DATASET_FILE):
    # define generator, generate questions
    dataset_generator = RagDatasetGenerator.from_documents(
        documents=documents,
        num_questions_per_chunk=10,  # set the number of questions per nodes
        transformations=[text_splitter],
    )

    # rag_dataset = dataset_generator.generate_questions_from_nodes()
    rag_dataset = await dataset_generator.agenerate_dataset_from_nodes()

    # save this dataset as it is required for the submission
    rag_dataset.save_json("rag_dataset.json")
else:
    import json
    with open(PERSIST_DATASET_FILE, "r") as f:
        # 將字典轉換為支持屬性訪問的結構
        rag_dataset = json.load(f, object_hook=lambda d: SimpleNamespace(**d))


In [16]:
for e in rag_dataset.examples:
    print(e)

namespace(query='Here are 10 questions based on the context information:', query_by=namespace(model_name='llama3.1:latest', type='ai'), reference_contexts=['# 校園行事曆\n\n# 一月\n\n1月15日：清新宿舍健康日\n\n地點：宿舍區健身房\n\n內容：免費健康檢查、瑜伽課程、營養講座\n\n# 二月\n\n2月5日：國際文化節\n\n地點：校園廣場\n\n內容：各國文化展覽、傳統美食、文化表演\n\n# 三月\n\n3月8日：環保義工日\n\n地點：校園及周邊社區\n\n內容：垃圾清理、植樹活動、環保教育\n\n3月15日：創業中心開放日\n\n地點：創業中心\n\n內容：創業指導、企業家講座、創業展示\n\n# 四月\n\n4月1日：清潔飲水與衛生設施宣導日\n\n地點：清潔飲水設施區\n\n內容：飲水衛生教育、節水設備展示\n\n4月20日：多元課程大樓開放日\n\n內容：課程介紹、學術講座、學生作品展覽\n\n# 五月\n\n5月5日：氣候變遷研討會\n\n地點：氣候變遷研究中心\n\n內容：氣候科學講座、政策討論、環保活動\n\n# 六月\n\n6月10日：全球發展合作論壇\n\n地點：國際合作中心\n\n內容：全球合作項目介紹、國際發展研討會\n\n6月25日：運動會\n\n地點：運動場\n\n內容：田徑比賽、團隊運動、健身挑戰\n\n# 七月\n\n7月20日：社區服務日\n\n地點：社區服務區\n\n內容：志工服務、社區活動、文化交流\n\n# 八月\n\n8月25日：二手市集\n\n地點：校園廣場\n\n內容：二手物品交易、回收教育'], reference_answer='I\'ll do my best to answer each question based on the provided context information. Please note that I won\'t have any prior knowledge or external information.\n\nHere are the answers:\n\n1. What is the date of

In [17]:
from llama_index.core.evaluation import (
    EvaluationResult,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
)
from llama_index.core import Response
import pandas as pd
import asyncio
from llama_index.core.llama_dataset import (
    LabelledRagDataExample,
)
# 累積評估結果的 DataFrame
eval_results_df = pd.DataFrame()


# 累積顯示評估結果的函數
def display_eval_df(
    response: Response,
    dataset_example: LabelledRagDataExample,
    relevancy_eval_result: EvaluationResult,
    correctness_eval_result: EvaluationResult,
) -> None:
    if not response.source_nodes:
        print("no response!")
        return
    eval_row = pd.DataFrame(
        [
            {   "Query": dataset_example.query,
                "Response": str(response),
                "Reference Answer": dataset_example.reference_answer,
                "Source": response.source_nodes[0].node.text[:1000] + "...",
                "Relevancy Eval Result": f"{'Pass' if relevancy_eval_result.passing else 'Fail'} \nscore: {relevancy_eval_result.score}",
                "Relevancy Reasoning": relevancy_eval_result.feedback,
                "Correctness Eval Result": f"{'Pass' if correctness_eval_result.passing else 'Fail'} \n\nscore: {correctness_eval_result.score}",
                "Correctness Reasoning": correctness_eval_result.feedback,
            }
        ]
    )
    global eval_results_df
    eval_results_df = pd.concat([eval_results_df, eval_row], ignore_index=True)


# 評估查詢引擎並顯示最終結果
async def evaluate_chat_engine(chat_engine, dataset_examples:LabelledRagDataExample):
    dataset_examples = dataset_examples[1:]
    questions = [e.query for e in dataset_examples]

    c = [chat_engine.achat(q) for q in questions]
    results = await asyncio.gather(*c)
    print("finished query")
    print('------------------------------')

    # 定義評估器
    relevancy_evaluator = RelevancyEvaluator()
    correctness_evaluator = CorrectnessEvaluator()

    relevancy_total_correct = 0
    correctness_total_correct = 0
    for response, dataset_example in zip(results, dataset_examples):
        relevancy_eval_result = relevancy_evaluator.evaluate_response(
            query=dataset_example.query, response=response
        )
        correctness_eval_result = correctness_evaluator.evaluate_response(
            query=dataset_example.query,
            response=response,
            reference=dataset_example.reference_answer,
        )

        # 累積每個查詢的評估結果
        display_eval_df(
            response, dataset_example, relevancy_eval_result, correctness_eval_result
        )

        if relevancy_eval_result.passing:
            relevancy_total_correct += 1
        if correctness_eval_result.passing:
            correctness_total_correct += 1

    return relevancy_total_correct, correctness_total_correct, len(results)


# 執行評估查詢並顯示最終累積結果
def run_evaluate_chat_engine(chat_engine, dataset_examples):
    relevancy_total_correct, correctness_total_correct, total_questions = asyncio.run(
        evaluate_chat_engine(chat_engine, dataset_examples)
    )
    global eval_results_df
    # styled_df = eval_results_df.style.set_properties(
    #     **{"inline-size": "400px", "overflow-wrap": "break-word"},
    #     subset=["Response", "Source"]
    # )
    styled_df = eval_results_df.style.set_properties(
        **{"white-space": "pre-wrap"}, 
    )

    display(styled_df)
    # 顯示總結結果
    print(f"Total Relevancy correct: {relevancy_total_correct} out of {total_questions}")
    print(
        f"Total Correctness correct: {correctness_total_correct} out of {total_questions}"
    )

In [18]:
# 執行查詢引擎並評估結果
run_evaluate_chat_engine(condense_plus_chat_engine, rag_dataset.examples)

Condensed question: What events are scheduled for the Research and Innovation Zone on November 8th?
Generated queries:
Here are three search queries related to the original input:
1. "Research and Innovation Zone schedule November 8"
2. "Events at Research and Innovation Zone on 11/8"
3. "November 8th events calendar Research and Innovation Zone"
Condensed question: Here is the rephrased follow-up question as a standalone question:

What other activities were listed in the school calendar for January?
Generated queries:
Here are three search queries related to your input:
1. "January school calendar events"
2. "School activities scheduled for January 2023"
3. "List of events and holidays in January school calendar"
Condensed question: Here is the rephrased standalone question:

What location will the campus energy-saving devices be exhibited on October 10th?
Generated queries:
Here are three search queries related to the input query:
1. "campus energy-saving device exhibition locations

,Query,Response,Reference Answer,Source,Relevancy Eval Result,Relevancy Reasoning,Correctness Eval Result,Correctness Reasoning
0,What is the title of the document?,"The title of the document is ""校園行事曆"" which means ""Campus Calendar"" in English.","The title of the document is ""# 校園行事曆"" which translates to ""# Campus Activity Calendar"".",# 校園行事曆 # 一月 1月15日：清新宿舍健康日 地點：宿舍區健身房 內容：免費健康檢查、瑜伽課程、營養講座 # 二月 2月5日：國際文化節 地點：校園廣場 內容：各國文化展覽、傳統美食、文化表演 # 三月 3月8日：環保義工日 地點：校園及周邊社區 內容：垃圾清理、植樹活動、環保教育 3月15日：創業中心開放日 地點：創業中心 內容：創業指導、企業家講座、創業展示 # 四月 4月1日：清潔飲水與衛生設施宣導日 地點：清潔飲水設施區 內容：飲水衛生教育、節水設備展示 4月20日：多元課程大樓開放日 內容：課程介紹、學術講座、學生作品展覽 # 五月 5月5日：氣候變遷研討會 地點：氣候變遷研究中心 內容：氣候科學講座、政策討論、環保活動 # 六月 6月10日：全球發展合作論壇 地點：國際合作中心 內容：全球合作項目介紹、國際發展研討會 6月25日：運動會 地點：運動場 內容：田徑比賽、團隊運動、健身挑戰 # 七月 7月20日：社區服務日 地點：社區服務區 內容：志工服務、社區活動、文化交流 # 八月 8月25日：二手市集 地點：校園廣場 內容：二手物品交易、回收教育...,Pass score: 1.0,"YES. The response ""The title of the document is '校園行事曆' which means 'Campus Calendar' in English."" is in line with the context information provided, which also mentions a ""campus calendar"" at the top.",Pass score: 4.0,"The generated answer is fully correct, as it accurately conveys the same information as the reference answer, including the Chinese character and its translation to English. The only difference is that the reference answer provides a more detailed translation (""# Campus Activity Calendar""), while the generated answer simply states ""Campus Calendar"", which is still accurate but less specific."
1,"On which date is the ""清新宿舍健康日"" event held?",根據校園行事曆，我們知道「清新宿舍健康日」是在1月15日舉辦的！,1月15日,# 校園行事曆 # 一月 1月15日：清新宿舍健康日 地點：宿舍區健身房 內容：免費健康檢查、瑜伽課程、營養講座 # 二月 2月5日：國際文化節 地點：校園廣場 內容：各國文化展覽、傳統美食、文化表演 # 三月 3月8日：環保義工日 地點：校園及周邊社區 內容：垃圾清理、植樹活動、環保教育 3月15日：創業中心開放日 地點：創業中心 內容：創業指導、企業家講座、創業展示 # 四月 4月1日：清潔飲水與衛生設施宣導日 地點：清潔飲水設施區 內容：飲水衛生教育、節水設備展示 4月20日：多元課程大樓開放日 內容：課程介紹、學術講座、學生作品展覽 # 五月 5月5日：氣候變遷研討會 地點：氣候變遷研究中心 內容：氣候科學講座、政策討論、環保活動 # 六月 6月10日：全球發展合作論壇 地點：國際合作中心 內容：全球合作項目介紹、國際發展研討會 6月25日：運動會 地點：運動場 內容：田徑比賽、團隊運動、健身挑戰 # 七月 7月20日：社區服務日 地點：社區服務區 內容：志工服務、社區活動、文化交流 # 八月 8月25日：二手市集 地點：校園廣場 內容：二手物品交易、回收教育...,Pass score: 1.0,"YES. The response for the query is in line with the context information provided. The response states that the ""清新宿舍健康日"" event is held on January 15th, which matches the date listed in the calendar under January.",Pass score: 4.0,"The generated answer is fully correct and relevant to the user query, with a slight difference in tone and style compared to the reference answer. The mention of ""校園行事曆"" (campus schedule) adds some context, but does not detract from the accuracy of the date."
2,"Where can students find more information about the ""創業中心開放日"" event?",根據校園行事曆，我們知道創業中心開放日的日期是3月15日，但並沒有提供更多關於這個活動的資訊。然而，文中提到該活動的地點是在創業中心，這可能意味著在創業中心可以找到更多有關這個活動的資訊。,"Based on the provided context information, it appears that the ""創業中心開放日"" (Entrepreneurship Center Open Day) event is scheduled to take place on March 15th. To find more information about this event, students can likely check the following locations: * The Entrepreneurship Center itself: This might be a good place to start looking for more details about the event. * The school's website or online calendar: It's possible that the school has listed more information about the event on their official website or online calendar. * Posters or flyers around campus: Students might have seen posters or flyers advertising the event, which could provide additional details. Unfortunately, there is no specific location mentioned in the provided context where students can find more information about the ""創業中心開放日"" event.",# 九月 # 9月5日：新學年開學典禮 地點：大禮堂 內容：校長致辭、新生介紹、校園生活指導 # 9月20日：學生權益講座 地點：學生權益保護機構 內容：學生權益保護講座、法律援助介紹 # 十月 # 10月10日：校園節能日 地點：校園各處 內容：節能設備展示、節能比賽 # 10月24日：國際交流活動 地點：國際合作中心 內容：留學生交流、國際文化展示 # 十一月 # 11月8日：科技創新大會 地點：科研與創新區 內容：科技創新展示、專題研討會 # 十二月 # 12月21日：聖誕慶祝活動 地點：校園廣場 內容：聖誕樹點燈、音樂表演、慈善義賣...,Pass score: 1.0,"YES. The response mentions the date of the ""創業中心開放日"" event (3月15日) and suggests that students can find more information about it at the v

Total Relevancy correct: 19 out of 19
Total Correctness correct: 16 out of 19
